In [49]:
from bayes_opt import BayesianOptimization
from bayes_opt.util import load_logs

from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, RBF
import json
import numpy as np
from itertools import product
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib.animation import FuncAnimation
%matplotlib inline
from IPython.display import HTML


In [58]:
gp =GaussianProcessRegressor(
            kernel=Matern(length_scale= 10),#RBF(length_scale=[0.05, 1]),
            alpha=1e-6,
            normalize_y=True,
            n_restarts_optimizer=5,
            random_state=5
        )

In [59]:
#Load data
log = "bad_logs.json"
x = []
y = []
with open(log, "r") as f:
    for line in f:
        line = json.loads(line)
        y.append(line["target"])
        x.append([line["params"]["a1"],line["params"]["t1"]])
x = np.array(x)

In [52]:
model = gp.fit(x,y)

In [60]:
label_x = "A1"
label_y = "T1"
bounds = [[0, 1.5],[300,1000]]

X1 = np.linspace(0, 1.5, 200)
X2 = np.linspace(300, 1000, 200)
x1, x2 = np.meshgrid(X1, X2)
X = np.hstack((x1.reshape(200*200,1),x2.reshape(200*200,1)))

fig = plt.figure(figsize=(13,5))


def update(i):
    fig.clear()
    ax1 = fig.add_subplot(121)
    ax2 = fig.add_subplot(122)
    
    gp.fit(x[:i],y[:i])
    m, v = gp.predict(X, return_std=True)

    cf1 = ax1.contourf(X1, X2, m.reshape(200,200),100)
    ax1.plot(x[:i-1,0], x[:i-1,1], 'r.', markersize=10, label=u'Observations')
    ax1.plot(x[i,0], x[i,1], 'r.', markersize=25, label=u'New Point')
    cb1 = fig.colorbar(cf1, ax=ax1)
    ax1.set_xlabel(label_x)
    ax1.set_ylabel(label_y)
    
    ax1.set_title('Posterior mean')
    ##
    ax2.plot(x[i,0], x[i,1], 'r.', markersize=25, label=u'New Point')
    ax2.plot(x[:i-1,0], x[:i-1,1], 'r.', markersize=10, label=u'Observations')
    cf2 = ax2.contourf(X1, X2, np.sqrt(v.reshape(200,200)),100)
    cb2 = fig.colorbar(cf2, ax=ax2)
    ax2.set_xlabel(label_x)
    ax2.set_ylabel(label_y)
    
    ax2.set_title('Posterior sd.')
    return ax1, ax2

<Figure size 936x360 with 0 Axes>

In [62]:
from mpl_toolkits import mplot3d
from matplotlib import cm

def update3d(i):
    fig.clear()
    ax1 = fig.add_subplot(121, projection='3d')
    ax2 = fig.add_subplot(122, projection='3d')
    
    gp.fit(x[:i],y[:i])
    m, v = gp.predict(X, return_std=True)
    ax1.plot_surface(X1, X2, m.reshape(200,200), 50, cmap=cm.coolwarm)
    ax1.set_xlabel(label_x)
    ax1.set_ylabel(label_y)
    ax1.set_zticks()
    ax1.set_title('Posterior mean')
    ##
    ax2.plot_surface(X1, X2, v.reshape(200,200), 50, cmap=cm.coolwarm)
    ax2.set_xlabel(label_x)
    ax2.set_ylabel(label_y)
    ax2.set_zticks()
    ax2.set_title('Posterior sd.')
    return ax1, ax2

In [63]:
import matplotlib.animation as animation
anim = FuncAnimation(fig, update, frames=np.arange(3, x.shape[0]), interval=500)
anim.save('line.gif', dpi=80, writer='imagemagick')
#plt.show()

In [64]:
HTML(anim.to_html5_video())

In [16]:
ax1 = fig.add_subplot(121, projection='3d')
ax2 = fig.add_subplot(122, projection='3d')

gp.fit(x[:10],y[:10])
m, v = gp.predict(X, return_std=True)
print(m.reshape(200,200).shape)
ax1.plot_surface(x1, x2, m.reshape(200,200), 50)
ax1.set_xlabel(label_x)
ax1.set_ylabel(label_y)
ax1.set_title('Posterior mean')
##
ax2.plot_surface(x1, x2, v.reshape(200,200), 50)
ax2.set_xlabel(label_x)
ax2.set_ylabel(label_y)
ax2.set_title('Posterior sd.')
plt.show

(200, 200)


TypeError: len() of unsized object

In [53]:
def prediction(a1, t1):
    m, std = model.predict([[a1, t1]], return_std=True)
    return m[0]

In [54]:
prediction(0.8, 730)

-4.904220017838696

In [57]:
from bayes_opt import BayesianOptimization
from bayes_opt.observer import JSONLogger, ScreenLogger
from bayes_opt.event import Events

logger = JSONLogger(path="bad_logs.json")

optimizer = BayesianOptimization(
    prediction,
    {'a1': (0, 1.5),
    't1': (300, 1000)})
optimizer.subscribe(Events.OPTMIZATION_STEP, logger)
optimizer.subscribe(Events.OPTMIZATION_STEP, ScreenLogger(verbose=2))
optimizer.set_gp_params(kernel=Matern(length_scale= 10))
optimizer.maximize( init_points=5,
    n_iter=30, acq="ucb", kappa=5.0)

|  1        | -12.82    |  1.11     |  330.2    |
|  2        | -6.401    |  0.9573   |  642.6    |
|  3        | -5.45     |  0.8537   |  959.8    |
|  4        | -11.03    |  0.8221   |  326.9    |
|  5        | -7.451    |  1.169    |  625.5    |
|  6        | -6.931    |  1.5      |  963.7    |
|  7        | -6.928    |  1.5      |  955.3    |
|  8        | -6.803    |  1.035    |  636.3    |
|  9        | -7.124    |  0.0      |  649.7    |
|  10       | -7.432    |  1.5      |  660.2    |
|  11       | -7.108    |  0.0      |  672.1    |
|  12       | -7.142    |  1.5      |  682.1    |
|  13       | -7.109    |  0.0      |  692.7    |
|  14       | -6.858    |  1.5      |  703.3    |
|  15       | -7.123    |  0.0      |  712.8    |
|  16       | -6.586    |  1.5      |  724.4    |
|  17       | -7.146    |  0.0      |  733.1    |
|  18       | -6.34     |  1.5      |  746.3    |
|  19       | -7.175    |  0.0      |  754.3    |
|  20       | -6.149    |  1.5      |  770.4    |
